# Pipeline for Epigenome-wide association studies in saliva samples
## Author: Jose Jaime Martinez-Magana

### Step 1 - Setting first environment
This pipeline uses idat files of DNA methylation arrays of saliva samples.
We recommend running this pipeline under a conda environment. Conda documentation (https://docs.conda.io/projects/conda/en/latest/user-guide/index.html).
The following steps describes how to install and setup the conda environment.

In [ ]:
# if you are using slurm job handler you could request resources with the following command
# uncomment the next line for job request
# srun --pty --mem=4G -p interactive bash
# if your server uses modules to handle packages and conda/anaconda/miniconda is install load your conda
# uncomment the next line if you have miniconda installed in your server
module load miniconda
# creating conda environment
# this will create an environment named ewas_saliva
conda create --name ewas_saliva
# conda activate environmnet
conda activate ewas_saliva

# install R via conda
conda install -c conda-forge r-base
# install bioconductor and devtools
conda install -c conda-forge r-devtools
# this will be requiered for installing several packages
conda install -c bioconda bioconductor-biocinstaller
# install a version of cmake and libxml, this package is requiered for compilation
conda install -c anaconda cmake
conda install -c anaconda libxml2
conda install -c conda-forge r-xml

# installing bioconductor dependencies
# enter R
R
# this will return an R prompt
# set path to the Rscript, this script will install the bioconductor packages
bioc_pack_path="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/script/environment/ewas_saliva/install.packages.bioconductor.R"
# this step will install all the bioconductor requiered packages
source(bioc_pack_path)
# in this sctep select the CRAN mirror close to you
# wait for packages to install

# installing CRAN packages
# set path to the Rscript, this script will install the CRAN packages
# change this path to the path of the package installer
# download the install.packages.Rscript from github: https://github.com/martinezjaime/ewas_saliva
Rscript_path="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/script/environment/install.packages.Rscript"
# install requiered packages
Rscript $Rscript_path

### this part of the code is needed because of problems with the dependencies between minfi and meffil
# suggestion added by Sheila Tiemi Nagamatsu: https://github.com/SheTiemi
# enter 
R
# reinstall preprocessCore with disable-threading
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("preprocessCore", configure.args="--disable-threading", force=TRUE)

# create a yaml file for the conda environment for replication of your analysis
conda env export > ewas_saliva_v1.0.0.yml

# once created exit the conda environmnet
conda deactivate
######################################################################################
# to enter to the environment type
# load conda if necesary
# uncomment the next command
# module load miniconda
conda activate ewas_saliva

### Step 2 - Setting second environment
This pipeline will use a glint conda environment to estimate epigenetic ancestry
The following steps describes how to install and setup the glint environment

In [ ]:
# set path for downloading glint
glint_path="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/script/environment/glint"
# move to that path
cd $glint_path
# download the last version of glint
# the next script only works in linux based systems
wget https://github.com/cozygene/glint/releases/download/1.0.4/GLINT_1.0.4.zip
# unzip the file
unzip GLINT_1.0.4.zip
# remove unnecesary file
rm -i GLINT_1.0.4.zip

# create a glint conda environment
# if you are using slurm job handler you could request resources with the following command
# uncomment the next line for job request
# srun --pty --mem=4G -p interactive bash
# if your server uses modules to handle packages and conda/anaconda/miniconda is install load your conda
# uncomment the next line if you have miniconda installed in your server
module load miniconda
# creating conda environment
# this will create an environment named glint
# glint requires python 2.x, create a conda environment with this version of python
conda create --name glint python=2.7
# conda activate environmnet
conda activate glint

# install glint dependencies
conda install -c anaconda numpy
conda install -c anaconda scipy
conda install -c anaconda scikit-learn
conda install -c anaconda pandas
conda install -c conda-forge matplotlib
# if the previous code does not work uncomment the next line
# pip install matplotlib --user
conda install -c anaconda statsmodels
conda install -c conda-forge cvxopt

# validate that all dependencies are installed
python validate_dependencies.py
# this script will print the next if everything is validated
# Validating all dependencies are installed...
# All dependencies are installed